# Dashboards Deep Dive: Parameters and Filters

Build interactive dashboards with dynamic parameters and filters using your semantic model.

## What You'll Learn

✅ Create static parameters for quick dropdown filters  
✅ Use query-based parameters for dynamic values  
✅ Add interactive filters to dashboards  
✅ Build visualizations from semantic models  

---

## Why Interactive Dashboards?

Your leadership needs dashboards ready by end of week. Basic static charts won't cut it - they need:
- **Filters by factory and region** - Different stakeholders need different views
- **Date range selection** - View last 7 days, 30 days, or custom ranges
- **Dynamic updates** - As new data arrives, dashboards auto-refresh

**Parameters and filters** make this possible without creating dozens of separate dashboards.

---

## Setup: Use Your Semantic Model

In the previous notebook, you created the `iot_sensor_metrics` semantic model. This dashboard will query that model, ensuring consistent metrics across all visualizations.

**Reference:**
- Table: `default.db_crash_course.iot_sensor_metrics`
- Metrics: `average_temperature`, `active_devices`, `total_sensor_readings`
- Dimensions: `factory_name`, `region`, `model_category`, `reading_date`

---

**References:**
- [Dashboard Parameters](https://docs.databricks.com/aws/en/dashboards/tutorials/query-based-params)
- [Dashboard Filters](https://docs.databricks.com/aws/en/dashboards/filters)
- [Creating Dashboards](https://docs.databricks.com/aws/en/dashboards/)


## 1. Edit a Dashboard

1. Click **Dashboards** in the sidebar
2. Select the dashboard from yesterday
3. Click **Edit Draft**
3. Name it: `{insert_your_username} - IoT Factory Performance`

## 2: Create Dynamic Filter

Now create a query that defines the factory filter values:

In [0]:
# Query: get_device_list
# This populates the dropdown dynamically

parameter_query = """
SELECT DISTINCT 
  `Device ID`
FROM dwx_airops_insights_platform_dev_workspace.db_crash_course.sensor_metrics
ORDER BY `Device ID`
"""

print("Parameter query for device dropdown:")
print(parameter_query)
print("\n📋 Copy this query into a new Dataset SQL query named 'get_device_list'")

In [0]:
# Query: model_performance_trend
# This populates the dropdown dynamically

model_performance_trend = """
SELECT
  model_id,
  COUNT(*) AS total_inspections,
  AVG(defect) AS avg_defect_rate,
  SUM(count) AS total_count,
  AVG(average_temperature) AS avg_temperature,
  AVG(average_density) AS avg_density,
  AVG(average_delay) AS avg_delay,
  AVG(average_rotation_speed) AS avg_rotation_speed,
  AVG(average_air_pressure) AS avg_air_pressure
FROM dwx_airops_insights_platform_dev_workspace.db_crash_course.inspection_gold
WHERE model_id = :model_id OR :model_id='All'
GROUP BY model_id
ORDER BY avg_defect_rate DESC
"""

print("Query to use the device dropdown:")
print(model_performance_trend)
print("\n📋 Copy this query into a new Dataset SQL query named 'model_performance_trend'")

##3. Link Parameter to Query

**In the Dashboard Page Editor:**

### Add Visualization
**Dataset**: model_performance_trend  
**Scatter Chart:**
- **X-axis**: `model_id`
- **Y-axis**: `AVG(avg_delay)`
- **Chart type**: Scatter

---

### Add a filter

- **Type**: Single Value
- **Fields**: get_model_list.model_id
- **Parameters**: model_performance_trend.model_id

You can now filter the chart based on the dynamic model_id values coming from the table!

##4: Use the Metric View

**In the Data Editor:**

### Add Dataset
- Click **+ Add data source**   
- Select **Metric Views**
- Select your metric view from the previous exercise


---

### Create Visualization

**In the Dashboard Page Editor:**
- Select the metric view dataset
- Ask Assistant to show `scatter plot of airflow rate over time`



**Click Publish!**